## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
#  Import and read the charity_data.csv.
import pandas as pd


# To display all rows
pd.set_option('display.max_rows', None)
# To display all columns
pd.set_option('display.max_columns', None)

df = pd.read_csv('/Users/phunghuynh/Desktop/NN ML/data.csv', delimiter=';')
df.head()

In [ ]:
df.info()

In [ ]:
df.rename(columns={"Daytime/evening attendance\t": "Attendance"}, inplace=True)
df.head()


In [ ]:
print(df['Target'].unique())

In [ ]:
# Create a reverse mapping dictionary
mapping = {"Dropout": 0, "Graduate": 1, "Enrolled": 2}

# Replace values in the "Target" column using the mapping
df["Target"] = df["Target"].map(mapping)

# Verify the changes by printing the first few rows
print(df.head())


In [ ]:
# Determine the number of unique values in each column.
print(df.nunique())

In [ ]:
# Look at Admission grade value counts to identify and replace with "Other"
# Admission_grade_counts = df['Admission grade'].value_counts()
# print(Admission_grade_counts)

In [ ]:
# # Choose a cutoff value and create a list of Admission grade to be replaced
# # use the variable name `Admission_grade_to_replace`
# Admission_grade_to_replace = Admission_grade_counts[Admission_grade_counts < 8].index.tolist()

# # Replace in dataframe
# for grade in Admission_grade_to_replace:
#     df['Admission grade'] = df['Admission grade'].replace(grade,"Other")

# # Check to make sure replacement was successful
# df['Admission grade'].value_counts()

In [ ]:
# Look at "Curricular units 1st sem (grade)" value counts to identify and replace with "Other"
# first_sem_grade_counts = df['Curricular units 1st sem (grade)'].value_counts()
# print(first_sem_grade_counts)

In [ ]:
# # look at Curricular units 1st sem (grade) value counts >1
# first_sem_grade_counts[first_sem_grade_counts > 1]

In [ ]:
# # Choose a cutoff value and create a list of "Curricular units 1st sem (grade)" to be replaced
# # use the variable name `first_sem_grade_to_replace`
# first_sem_grade_to_replace = first_sem_grade_counts[first_sem_grade_counts < 100].index.tolist()
# # Replace in dataframe
# for first_sem_grade in first_sem_grade_to_replace:
#     df['Curricular units 1st sem (grade)'] = df['Curricular units 1st sem (grade)'].replace(first_sem_grade,"Other")

# # Check to make sure replacement was successful
# df['Curricular units 1st sem (grade)'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df, drop_first=True)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = df['Target'].values
X = df.drop(['Target'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 24
hidden_nodes_layer3 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='tanh'))
nn.add(Dropout(0.5))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='tanh'))
nn.add(Dropout(0.5))  # Dropout layer with 50% rate
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='tanh'))
nn.add(Dropout(0.5))  # Dropout layer with 50% rate
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=500, callbacks=[early_stopping])


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")